In [ ]:
# Development Plan

In [ ]:
# Load training data into working directory:
# For training use the Cornell Movie-Dialog Corpus at path = kagglehub.dataset_download("rajathmc/cornell-moviedialog-corpus") 
#                           raw_scripts_urls.txt, 
#                           movie_titles_metadata.txt, 
#                           movie_lines.txt, 
#                           movie_conversations.txt, and 
#                           movie_characters_metadata.txt

# Next, read the relevant files directly and preprocess them to train the chatbot.

# The most important files for generating dialogues are movie_lines.txt and movie_conversations.txt, as they contain the actual dialogues and 
# conversation pairings.

# The chatbot implementation consists of:

#    Preprocessing:
#        The movie_lines.txt and movie_conversations.txt files are used to create input-output pairs for training the chatbot.
#        The data is Tokenized using a transformer-compatible tokenizer.

#    Model Training:
#        Transformer-based model GPT-2 is used for fine-tuning.

#    Multi-turn Conversations:
#        The conversation context is maintained to handle multi-turn dialogues.

#    Web Interface:
#        Gradio is used to build a web interface for user interaction.

In [ ]:
# Preprocessing the Data

In [ ]:
#    the movie_lines.txt and movie_conversations.txt will be read to extract dialogues and conversation pairs

In [1]:
import pandas as pd

# Paths to the files
movie_lines_path = './movie_lines.txt'
movie_conversations_path = './movie_conversations.txt'

# Step 1: Load movie lines into a dictionary
id2line = {}
with open(movie_lines_path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        parts = line.strip().split(' +++$+++ ')
        if len(parts) == 5:
            id2line[parts[0]] = parts[4]  # parts[0] is the line ID, parts[4] is the actual line text

# Step 2: Load movie conversations and create conversation pairs
conversations = []
with open(movie_conversations_path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        parts = line.strip().split(' +++$+++ ')
        if len(parts) == 4:
            # Extract line IDs from the conversation
            line_ids = eval(parts[3])
            for i in range(len(line_ids) - 1):
                # Create pairs of (input, output) for conversation
                input_line = id2line.get(line_ids[i], "")
                target_line = id2line.get(line_ids[i + 1], "")
                if input_line and target_line:
                    conversations.append((input_line, target_line))

# Print out a few examples
for i in range(5):
    print(f"Input: {conversations[i][0]}")
    print(f"Target: {conversations[i][1]}")
    print()

Input: Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Target: Well, I thought we'd start with pronunciation, if that's okay with you.

Input: Well, I thought we'd start with pronunciation, if that's okay with you.
Target: Not the hacking and gagging and spitting part.  Please.

Input: Not the hacking and gagging and spitting part.  Please.
Target: Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

Input: You're asking me out.  That's so cute. What's your name again?
Target: Forget it.

Input: No, no, it's my fault -- we didn't have a proper introduction ---
Target: Cameron.



In [ ]:
# Fine-Tuning a Transformer Model (GPT-2)

In [ ]:
#    Now, having the conversation pairs, the pre-trained model can be fine-tuned like GPT-2

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
# Initialize the tokenizer and model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [4]:
# Use eos_token as the pad_token
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [6]:
class MovieDataset(Dataset):
    def __init__(self, conversations, tokenizer, max_length=512):
        self.conversations = conversations
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        input_text, target_text = self.conversations[idx]
        
        # Tokenize input and target text, with eos token and padding
        encoding = self.tokenizer(f"{input_text} {self.tokenizer.eos_token}", 
                                  f"{target_text} {self.tokenizer.eos_token}", 
                                  return_tensors='pt', 
                                  max_length=self.max_length, 
                                  truncation=True, 
                                  padding='max_length')
        
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        # In language modeling, input_ids are also used as labels
        labels = input_ids.clone()  # Copy of input_ids for labels

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels  # Include labels in the returned batch
        }

In [7]:
# Prepare the dataset and dataloader
dataset = MovieDataset(conversations, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [8]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\josep\anaconda3b\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader

# Initialize the tokenizer and smaller model (use 'gpt2' for the smallest model)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # Use 'gpt2'
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')  # Use 'gpt2'

In [15]:
# Tokenize the conversation pairs
class MovieDataset(Dataset):
    def __init__(self, conversations, tokenizer, max_length=512):
        self.conversations = conversations
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        input_text, target_text = self.conversations[idx]
        
        # Tokenize input and target text, with eos token and padding
        encoding = self.tokenizer(f"{input_text} {self.tokenizer.eos_token}", 
                                  f"{target_text} {self.tokenizer.eos_token}", 
                                  return_tensors='pt', 
                                  max_length=self.max_length, 
                                  truncation=True, 
                                  padding='max_length')
        
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        # In language modeling, input_ids are also used as labels
        labels = input_ids.clone()  # Copy of input_ids for labels

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels  # Include labels in the returned batch
        }

# Prepare the dataset and dataloader with smaller batch size
dataset = MovieDataset(conversations, tokenizer)
batch_size = 8  # Reduce batch size to avoid memory issues
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizer with moderate learning rate
learning_rate = 5e-5  # Conservative learning rate
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Scheduler for learning rate decay
num_training_steps = len(dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()

C:\Users\josep\AppData\Local\Temp\ipykernel_19068\4023032304.py:48: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [16]:
import random
from torch.utils.data import Subset

# Assuming `conversations` is your full list of conversation pairs
total_samples = len(conversations)  # This would be 305,000
subset_size = 3000  # Target smaller size

# Randomly select 3,000 indices from the full dataset
subset_indices = random.sample(range(total_samples), subset_size)

# Use Subset to create a smaller dataset from the full dataset
small_dataset = Subset(dataset, subset_indices)

# Now create a DataLoader for the smaller dataset
small_dataloader = DataLoader(small_dataset, batch_size=8, shuffle=True)


In [17]:
from tqdm import tqdm  # For tracking progress

In [21]:
# Training loop
for epoch in range(1):  # 1 epoch for demo purposes
    with tqdm(total=len(small_dataloader), desc=f"Epoch {epoch + 1}") as pbar:
        for batch in small_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # Fetch the labels
            
            # Pass the labels explicitly to the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)            
            #outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch + 1} loss: {loss.item()}")
            pbar.update(1)  # Update progress bar after each batch
    
    print(f"Epoch {epoch + 1} loss: {loss.item()}")

Epoch 1:   0%|▏                                                                    | 1/375 [02:02<12:45:52, 122.87s/it]

Epoch 1 loss: 0.6645388007164001


Epoch 1:   1%|▍                                                                      | 2/375 [02:58<8:36:54, 83.15s/it]

Epoch 1 loss: 0.3914414048194885


Epoch 1:   1%|▌                                                                      | 3/375 [03:59<7:33:03, 73.08s/it]

...

Epoch 1 loss: 0.21482917666435242


Epoch 1: 100%|████████████████████████████████████████████████████████████████████▊| 374/375 [7:47:18<01:16, 76.04s/it]

Epoch 1 loss: 0.23318830132484436


Epoch 1: 100%|█████████████████████████████████████████████████████████████████████| 375/375 [7:48:39<00:00, 74.98s/it]

Epoch 1 loss: 0.25335681438446045
Epoch 1 loss: 0.25335681438446045


In [22]:
# Save the fine-tuned model
model.save_pretrained('./movie_chatbot_model')
tokenizer.save_pretrained('./movie_chatbot_tokenizer')

('./movie_chatbot_tokenizer\\tokenizer_config.json',
 './movie_chatbot_tokenizer\\special_tokens_map.json',
 './movie_chatbot_tokenizer\\vocab.json',
 './movie_chatbot_tokenizer\\merges.txt',
 './movie_chatbot_tokenizer\\added_tokens.json')

In [ ]:
# Multi-Turn Conversation with Context

In [ ]:
#     handling multi-turn conversations by storing conversation history

In [23]:
def generate_response(model, tokenizer, conversation_history, max_length=512):
    input_text = " ".join(conversation_history) + tokenizer.eos_token
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

In [ ]:
# Web Interface using Gradio

In [ ]:
#     using Gradio to build a simple web interface

In [25]:
import gradio as gr

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./movie_chatbot_model')
tokenizer = GPT2Tokenizer.from_pretrained('./movie_chatbot_tokenizer')

conversation_history = []

def chatbot(user_input):
    global conversation_history
    conversation_history.append(user_input)
    response = generate_response(model, tokenizer, conversation_history)
    conversation_history.append(response)
    return response

# Build the Gradio interface
gr.Interface(fn=chatbot, inputs="text", outputs="text").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Evaluation 

In [ ]:
#    Evaluate the chatbot using BLEU for generated responses

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def evaluate_bleu(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference.lower())
    candidate_tokens = nltk.word_tokenize(candidate.lower())
    score = sentence_bleu([reference_tokens], candidate_tokens)
    return score

In [29]:
import torch
import math
from tqdm import tqdm  # Progress bar library
from nltk.translate.bleu_score import sentence_bleu

def evaluate(model, small_dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        # Use tqdm to wrap the dataloader for a progress bar
        for batch in tqdm(small_dataloader, desc="Evaluating", leave=False):
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
            total_loss += outputs.loss.item()
    
    avg_loss = total_loss / len(small_dataloader)
    perplexity = math.exp(avg_loss)
    return perplexity

# Calculate BLEU score for evaluation
def bleu_score(reference, candidate):
    return sentence_bleu([reference.split()], candidate.split())

# Example usage for evaluation with progress bar
perplexity = evaluate(model, small_dataloader)
print(f'Perplexity: {perplexity}')

Perplexity: 1.2213929358027669


In [ ]:
# Summary

In [ ]:
#    Preprocessing: Loaded movie_lines.txt and movie_conversations.txt, tokenized dialogues.
#    Model: Fine-tuned GPT-2 for conversational modeling.
#    Context Management: Maintained conversation history for multi-turn interactions.
#    Web Interface: Used Gradio for a simple chatbot interface.
#    Evaluation: Added BLEU score as a metric.

#This implementation enables a transformer-based chatbot that handles multi-turn conversations, adapts to context, and uses the loaded data